In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('seedfinder').getOrCreate()

22/06/14 10:50:44 WARN Utils: Your hostname, mehrdad-Standard resolves to a loopback address: 127.0.1.1; using 128.179.176.230 instead (on interface wlp4s0)
22/06/14 10:50:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 10:50:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 10:50:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/14 10:50:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/14 10:50:45 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/06/14 10:50:45 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/06/

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

### Explore Data

In [ ]:
data = spark.read.csv('../data/seeds_dataset.csv', inferSchema=True, header=True)

In [ ]:
data.printSchema()

In [ ]:
data.head(1)

In [ ]:
data.count()

In [ ]:
# we know there are 3 different kinds of wheat, so we use K = 3

In [ ]:
from pyspark.ml.clustering import KMeans

### Create Feature Set

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols=data.columns,outputCol='features')

In [ ]:
with_features = assembler.transform(data).select('features')

### Feature Scaling

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

In [ ]:
scalar_model = scaler.fit(with_features)

In [ ]:
scaled_data = scalar_model.transform(with_features)

In [ ]:
scaled_data.select('scaled_features').head(1)

### Train KMeans Model

In [ ]:
kmeans = KMeans(featuresCol='scaled_features', k=3)
model = kmeans.fit(scaled_data)

### Interpret Cluster Results

In [ ]:
model.computeCost(scaled_data)

In [ ]:
model.clusterCenters()

In [ ]:
results = model.transform(scaled_data)

In [ ]:
results.select('prediction').show()